In [55]:

# Dependencies

import pandas as pd
import numpy as np
import scipy.stats as stats
from pathlib import Path
from py_scripts.functions import Master_NC_Dataframe
from py_scripts.functions import column_list
from py_scripts.functions import data_categories


#Specific to the Chi Square Test
# Quick install to dev env with pip install researchpy
import researchpy as rp


In [56]:
race_and_infant_death = data_categories['per_1000']['Female_Other_Age'] + data_categories['per_1000']['Female_White_Age'] + data_categories['per_1000']['Poverty and Welfare'] + data_categories['per_1000']['Life Events and Family Planning']
racebyinfant_df = Master_NC_Dataframe[race_and_infant_death]




racebyinfant_df = racebyinfant_df.filter(items = [91, 59, 40, 33, 25], axis=0)

racebyinfant_df

,Female-Other-Population Age 0-4,Female-Other-Population Age 5,Female-Other-Population Age 6-9,Female-Other-Population Age 10-12,Female-Other-Population Age 13,Female-Other-Population Age 14,Female-Other-Population Age 15,Female-Other-Population Age 16-17,Female-Other-Population Age 18,Female-Other-Population Age 19,...,White Persons for Whom Poverty Status Is Determined,White Persons in Poverty,Divorces,Marriages,Pregnancies for Females 15-19,Pregnancies for Females of All Ages,Reported Abortions,Substantiated Reports of Child Abuse,Substantiated Reports of Child Abuse and Neglect,Substantiated Reports of Child Neglect
91,11946.0,2497.0,10304.0,8219.0,2981.0,2960.0,3016.0,5999.0,2920.0,2969.0,...,681801.0,41457.0,2.0,4.0,606.0,15921.0,3313.0,13,53,102
59,17620.0,3531.0,13618.0,10092.0,3453.0,3378.0,3392.0,6785.0,3352.0,3420.0,...,559916.0,40657.0,2.0,2.0,915.0,19326.0,4569.0,34,26,188
40,8014.0,1591.0,6288.0,4848.0,1676.0,1668.0,1737.0,3413.0,2318.0,2892.0,...,279667.0,26870.0,1.0,2.0,379.0,8005.0,2023.0,22,36,109
33,4448.0,951.0,3978.0,3007.0,1068.0,967.0,1010.0,1905.0,1094.0,1216.0,...,237910.0,28969.0,1.0,1.0,315.0,5401.0,1083.0,15,6,31
25,6555.0,1251.0,4956.0,4057.0,1340.0,1357.0,1263.0,2516.0,1327.0,1354.0,...,152224.0,21133.0,1.0,1.0,339.0,6631.0,1475.0,23,40,92


In [70]:

finalinfantdf = racebyinfant_df.T
#finalinfantdf = finalinfantdf.rename({'91': 'Wake', '59': 'Mecklenburg', '40': 'Guilford', '33': 'Forsyth', '25':'Cumberland'}, axis = 1, inplace = True)
finalinfantdf.columns = ['Wake', 'Mecklenburg', 'Guilford', 'Forsyth', 'Cumberland']

finalinfantdf= finalinfantdf.T


In [154]:


Female_Demo = { 'Counties' : ['Wake County', 'Mecklenburg County', 'Guilford County', 'Forsyth County', 'Cumberland County'],
                  'Percentages (White)': [0.571*0.517, 0.447*0.519, 0.472*0.527, 0.544*0.526, 0.398*0.505],
                  'Percentages (Black)': [0.181*0.517, 0.291*0.519, 0.331*0.527, 0.245*0.526, 0.371*0.505],
                  'White Population': [1197936*0.571*0.517, 1174237*0.447*0.519, 557166*0.472*0.527,
                                       392166*0.544*0.526, 339318*0.398*0.505],
                  'Black Population': [1197936*0.181*0.517, 1174237*0.291*0.519, 557166*0.331*0.527, 
                                       392166*0.245*0.526, 339318*0.371*0.505],
                  }

Female_top5_df = pd.DataFrame(Female_Demo).round(3)

Female_top5_df

,Counties,Percentages (White),Percentages (Black),White Population,Black Population
0,Wake County,0.295,0.094,353639.093,112099.257
1,Mecklenburg County,0.232,0.151,272414.764,177343.840
2,Guilford County,0.249,0.174,138591.700,97190.366
3,Forsyth County,0.286,0.129,112215.948,50538.432
4,Cumberland County,0.201,0.187,68199.525,63572.924


In [155]:
whiteblack_df = finalinfantdf[['White Persons in Poverty', 'Black Persons in Poverty']]

hello = whiteblack_df.reset_index()

hello2 = hello.rename(columns={'index':'Counties'})

hello2.replace('Wake', 'Wake County', inplace=True)
hello2.replace('Mecklenburg', 'Mecklenburg County', inplace=True)   
hello2.replace('Guilford', 'Guilford County', inplace=True) 
hello2.replace('Forsyth', 'Forsyth County', inplace=True)   
hello2.replace('Cumberland', 'Cumberland County', inplace=True)

poverty_db = pd.DataFrame.merge(Female_top5_df, hello2, how = 'outer', on='Counties')

poverty_db = poverty_db.set_index('Counties')
poverty_db['White Persons in Poverty'] = poverty_db['White Persons in Poverty'].astype(float)
poverty_db['Black Persons in Poverty'] = poverty_db['Black Persons in Poverty'].astype(float)
poverty_db['White Population'] = poverty_db['White Population'].astype(float)
poverty_db['Black Population'] = poverty_db['Black Population'].astype(float)

poverty_db['Percentage White in Poverty'] = poverty_db['White Persons in Poverty'] / poverty_db['White Population']
poverty_db['Percentage Black in Poverty'] = poverty_db['Black Persons in Poverty'] / poverty_db['Black Population']

poverty_db

,Percentages (White),Percentages (Black),White Population,Black Population,White Persons in Poverty,Black Persons in Poverty,Percentage White in Poverty,Percentage Black in Poverty
Counties,,,,,,,,
Wake County,0.295,0.094,353639.093,112099.257,41457.0,28820.0,0.117230,0.257094
Mecklenburg County,0.232,0.151,272414.764,177343.840,40657.0,47419.0,0.149247,0.267385
Guilford County,0.249,0.174,138591.700,97190.366,26870.0,39837.0,0.193879,0.409886
Forsyth County,0.286,0.129,112215.948,50538.432,28969.0,22023.0,0.258154,0.435767
Cumberland County,0.201,0.187,68199.525,63572.924,21133.0,28593.0,0.309870,0.449767


In [164]:


crosstab = pd.crosstab(poverty_db['Counties'], poverty_db['Percentages (White)'], poverty_db['Percentage White in Poverty'])

crosstab

KeyError: 'Counties'

In [160]:
# Second method is to use researchpy package
(stats.chi2_contingency(crosstab))



Chi2ContingencyResult(statistic=20.000000000000004, pvalue=0.22022064660169877, dof=16, expected_freq=array([[0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2]]))

In [ ]:
critical_value = stats.chi2.ppf(q = 0.95, df = 2)

stats.chisquare(df['observed'], df['expected'])